# Outline

## North

- ลบ note หรือ comment กำกับข้อมูลใน column 4-5
- ใช้ regex ลบ ดู, ก็ว่า, ก็เรียก

## South

- มีการใช้เลขไทย [๑., ๑).] เพื่อแสดง POS ที่หลากหลาย
- "อาลปนะ.", "บุรพ."
- "แสลง." instead of "POS"

## Isan

- "ดู" pattern
- remove any note or comment in 5th column
- clear the first row

## Central

# Main Program

## Load the Data

In [ ]:
# downloading packages

# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
# !pip3 install openpyxl

In [ ]:
# loading the data

import pandas as pd
import numpy as np
import os
import re

# north_path = r"datasets\new_พจนานุกรมภาษาถิ่นเหนือ.xlsx"
# south_path = r"datasets\new_พจนานุกรมภาษาถิ่นใต้.xlsx"
isan_path = r"datasets\new_พจนานุกรมภาษาถิ่นอีสาน.xlsx"
orst_path = r"datasets\ราชบัณฑิตฯ"

def load_sheets(file_path, exclude_sheets):
    all_sheets = pd.ExcelFile(file_path).sheet_names
    importing_sheets = [sheet for sheet in all_sheets if sheet not in exclude_sheets]
    return pd.read_excel(file_path, sheet_name=importing_sheets, header=None)

# north_df = pd.read_excel(north_path, sheet_name=None, header=None)
# south_df = pd.read_excel(south_path, sheet_name=None, header=None)
isan_df = pd.read_excel(isan_path, sheet_name=None, header=None)

def load_excel(directory):
    excel_files = {}
    for filename in os.listdir(directory):
        name = os.path.splitext(filename)[0]
        excel_files[name] = pd.read_excel(os.path.join(directory, filename), header=None)
    return excel_files

orst_df = load_excel(orst_path)

## Inspect the Data

In [ ]:
# run to copy

# north = north_df.copy()
# south = south_df.copy()
isan = isan_df.copy()
orst = orst_df.copy()

In [ ]:
# inspect the data
def inspect(num=10, *dfs):
  for df in dfs:
    for key in df.keys():
      print(f"{key}\n{df[key].head(num)}\n")

inspect(5, isan, orst)
# inspect(10, north, south)

In [ ]:
north["บ"]

## Cleaning

In [ ]:
# cleaning

# make dataframe two columns
def cleaning(*dfs):
    for df in dfs:
        removed = [
            'ช่อง 1', 'ลูกคำ/ ความหมาย', 'แม่คำ', 'ความหมายลูกคำ คำสื่อ',
            'headword', '2', '3']

        for key in df.keys():
            # if df == north:
            #     df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
            #     df[key] = df[key].dropna(how='all')
            #     df[key] = df[key].drop(df[key].columns[2:], axis=1)
            #     df[key] = df[key].fillna("")
            #     df[key].columns = ['word', 'meaning']

            # elif df == south:
            #     df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
            #     df[key] = df[key].dropna(how='all')
            #     df[key] = df[key].drop(df[key].columns[2:], axis=1)
            #     df[key] = df[key].fillna("")
            #     df[key].columns = ['word', 'meaning']
            
            if df == isan:
                df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].drop(df[key].columns[2:], axis=1)
                df[key] = df[key].fillna("")
                df[key].columns = ['word', 'meaning']

            elif df == orst:
                df[key] = df[key].iloc[:, :-1]
                df[key].iloc[:, 0] = df[key].apply(lambda row: row.iloc[1] if pd.isnull(row.iloc[0]) else row.iloc[0], axis=1)
                df[key] = df[key].drop(columns=[df[key].columns[1]])
                df[key] = df[key].reset_index(drop=True)
                df[key].columns = ['word', 'meaning']
                df[key] = df[key].iloc[1:]

# cleaning(north, south)
cleaning(isan, orst)

## Checking

In [ ]:
# checking
def checking(df):
    comma_value = []
    missing_value = []
    POS_value = []

    regex = r',(?!(?:[^(]*\)))'
    POS = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|บุ\.|\*ส\.|สำ\.|\*สำ\.|ล\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|สรรพ\.|อ\.|อุทาน\.|\*อ\.|อุ\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|\?)(.*)$"

    # checking comma in the first column
    for key in df.keys():
        for value in df[key]['word']:
            if re.search(regex, value):
                comma_value.append(value)
        
        for value in df[key]['meaning']:
            matches = re.findall(POS, value)
            if len(matches) > 1:
                POS_value.append(value)
            elif len(matches) < 1 and value != "" and value.isspace() == False:
                print(value)

    print(comma_value if len(comma_value) > 0 else None)
    print(POS_value if len(comma_value) > 0 else None)

# checking(north)
# checking(south)
checking(isan)
# checking(orst)

In [ ]:
# inspect(10, north, south)
inspect(10, isan, orst)

## Split POS

In [ ]:
# splitting POS

def rearrange(df):
  for key in df.keys():
    pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
    
    df[key]["คำ"] = df[key]['word']
    df[key]["POS"] = ""
    df[key]["ความหมาย"] = ""

    if df[key]['meaning'].str.startswith((
      "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.", "สำ.", "ส.", "*ส.", "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.",
      "?", "บุพ.", "บ.", "บุ.", "บุรพ.", "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.", "อุทาน.", "คำลงท้าย.")).any():
    
      df[key][["POS", "ความหมาย"]] = df[key]['meaning'].str.extract(pattern, expand=True)
      df[key]["ความหมาย"] = df[key].apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

    # fill no POS
    pos_mask = df[key]['meaning'].str.match(pattern, na=False)
    df[key]["POS"] = df[key]["POS"].fillna("")
    df[key].loc[~pos_mask, "ความหมาย"] = df[key].loc[~pos_mask, 'meaning']

    # remove first two columns
    df[key] = df[key].iloc[:, 2:]

# rearrange(north)
# rearrange(south)
rearrange(isan)
rearrange(orst)

def clean_whitespace(df):
  for key in df.keys():
    for column in df[key].columns:
      if df[key][column].dtype == 'object':
        df[key][column] = df[key][column].apply(
          lambda text: ' '.join(text.split()) if isinstance(text, str) else text
        )
  return df

# north = clean_whitespace(north)
# south = clean_whitespace(south)
isan = clean_whitespace(isan)
orst = clean_whitespace(orst)

def no_whitespace(df):
  for key in df.keys():
      df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

# no_whitespace(north)
# no_whitespace(south)
no_whitespace(isan)
no_whitespace(orst)

In [ ]:
# orst splitting

for key in orst.keys():
    # split semi-colon in the third column
    orst[key] = orst[key].assign(ความหมาย=orst[key]['ความหมาย'].str.split(';')).explode('ความหมาย')
    orst[key]['ความหมาย'] = orst[key]['ความหมาย'].str.strip()

    # filling and reseting index
    orst[key] = orst[key].ffill()
    orst[key] = orst[key].reset_index(drop=True)

inspect(10, orst)

## Fill Blank Values

In [ ]:
def fill_blank_words(iter=2, *dfs):
    for df in dfs:
        for _ in range(iter):
            for key in df.keys():
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].infer_objects(copy=False)
                df[key] = df[key].replace(r'^\s*$', np.nan, regex=True) 
                df[key] = df[key].ffill()
                df[key].drop_duplicates(inplace=True)
                df[key] = df[key].reset_index(drop=True)
        return df

# fill_blank_words(2, north)
# fill_blank_words(2, south)
fill_blank_words(2, isan)
fill_blank_words(2, orst)

In [ ]:
import pandas as pd

path = r"datasets\cleaned_พจนานุกรมภาษาถิ่นใต้.xlsx"

df = pd.read_excel(path, sheet_name=None)

In [ ]:
df = df.copy()

In [ ]:
df

In [ ]:
import re

def no_more_whitespace(df):
    for key in df.keys():
        df[key].drop_duplicates(inplace=True)
        df[key] = df[key].reset_index(drop=True)
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

no_more_whitespace(df)
# no_more_whitespace(isan)
# no_more_whitespace(orst)

In [ ]:
south = df

In [ ]:
south

## Output

In [ ]:
# save as csv

def df_to_csv(dfs, region, path):
    for sheet, df in dfs.items():
        file = f"{path}/{region}_{sheet}.csv"
        df.to_csv(file, index=False, encoding='utf-8-sig')

regions = {
    # 'north': north,
    # 'cen': orst,
    'south': south,
    # 'isan': isan
    }

for region, dfs in regions.items():
    df_to_csv(dfs, region, r"cleaned_พจนานุกรมภาษาถิ่น\พจนานุกรมภาษาถิ่นใต้")

In [ ]:
# save as xlsx file

def df_to_xlsx(dfs, region, path):
    with pd.ExcelWriter(f"{path}/{region}.xlsx") as writer:
        for sheet, df in dfs.items():
            df.to_excel(writer, sheet_name=sheet, index=False)

regions = {
    # 'cleaned_พจนานุกรมภาษาถิ่นเหนือ': north,
    # 'cleaned_พจนานุกรมภาษาถิ่นใต้': south,
    'cleaned_พจนานุกรมภาษาถิ่นอีสาน': isan,
    'cleaned_พจนานุกรมภาษาถิ่นกลาง': orst
    }

for region, dfs in regions.items():
    df_to_xlsx(dfs, region, r"output")


# Test

In [ ]:
d = {'col1': [1, 2],
     'meaning': ['​ ​แก้วสามประการ หมายถึง พระพุทธ พระธรรม พระสงฆ์', '​ เป็นช่อง กว้าง  ใหญ่ เห็นได้จะ แจ้ง  ']}
df = pd.DataFrame(data=d)

# Using enumerate to modify the DataFrame directly
for i, value in enumerate(df['meaning']):
    df.loc[i, 'meaning'] = re.sub(r"^\s+|\s+$", "", value.replace("\u200b", ""))

df.head()